In [1]:
#Ensuring compliance of code with both python2 and python3
#!pip install ipyparallel
#!pip install jpype1
#!pip install SALib
#!pip install pyNetLogo
from __future__ import division, print_function
try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pyNetLogo

#Import the sampling and analysis modules for a Sobol variance-based 
#sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol
problem = { 
  'num_vars': 6,
  'names': ['random-seed',
            'grass-regrowth-time',
            'sheep-gain-from-food',
            'wolf-gain-from-food',
            'sheep-reproduce',
            'wolf-reproduce'], 
  'bounds': [[1, 100000],
             [20., 40.], 
             [2., 8.], 
             [16., 32.],
             [2., 8.],
             [2., 8.]]
}
n = 1000
param_values = saltelli.sample(problem, n, calc_second_order=True)
param_values.shape
#!pip install nl4py
import os
import ipyparallel

client = ipyparallel.Client()
print(client.ids)
direct_view = client[:]
import os
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))
#Push the "problem" variable from the notebook to a corresponding variable on the engines
direct_view.push(dict(problem=problem))

[0, 1, 2, 3]


<AsyncResult: _push>

In [2]:
%%px 

import os
os.chdir(cwd)
import time
import pyNetLogo
import pandas as pd
netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home = "C:\Program Files\NetLogo 6.0.3", netlogo_version = '6')
netlogo.load_model('Wolf Sheep Predation.nlogo')

In [3]:
def simulation(experiment):
    
    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    counts = netlogo.repeat_report(['ticks','count sheep','count wolves'], 100)    
    
    results = pd.Series([counts.shape[0],
                         counts.iloc[-1:1], 
                         counts.iloc[-1:2]],index=['ticks','Avg. sheep', 'Avg. wolves'])
    
    return results

In [4]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

59074


In [11]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

43980


In [13]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

37859


In [14]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

38855


In [9]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

38617


In [10]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

36664


In [11]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

ValueError: DataFrame constructor not properly called!

In [ ]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

In [ ]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

In [ ]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

In [47]:
results.to_csv('Sobol_parallel.csv')

In [23]:
results.head(5)

,Avg. sheep,Avg. wolves
0,133.39,110.63
1,137.45,129.16
2,133.39,110.63
3,133.39,110.63
4,294.98,92.50
